In [2]:
from dataclasses import dataclass

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir = str
    source_url = str
    local_data_file = str
    unzip_dir = str

In [9]:
from recSys.constants import *
from recSys.utils.common import read_yaml, create_directories

In [8]:
import os
print(os.getcwd())

/home/shekhar/ml_projects/recSys/research


In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_path = CONFIG_FILE_PATH,
        params_path = PARAMS_FILE_PATH,
        schema_path = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        self.schema = read_yaml(schema_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config

In [15]:
from recSys import logger
from recSys.utils.common import get_size
import zipfile
from urllib import request

In [ ]:
class DataIngestion:
    def __init__(self, config : DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded!!! with the details \n{headers}")
        else:
            logger.info(f"File already exists of size : {get_size(self.config.local_data_file)}")

    def extract_zip(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as z:
            z.extractall(unzip_path)